In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.linalg import expm
import math

import plotly.express as px
import plotly.graph_objects as go

## Scan Matching using Iterative Closest Point (ICP) algorithm

In [10]:
def estimateCorrespondence(xSet,ySet,t,rotMat,dMax):
    '''
    A function to find the points in xSet that corresponds to the same in ySet.

        Parameters:
            xSet (np.matrix): Point cloud
            ySet(np.matrix): Another Point cloud
            t (np.matrix): Initial guess of translation matrix
            rotMat(np.matrix): Initial guess of rotation matrix
            dMax(float): Maximum admissiable distance between each point in xSet and ySet
        Return:
            C (list): Correspondance list
    '''
    C =[]
    
    for i in range(len(xSet)):
        
        # Find the diffrence between points in Y set and the transformed point in X set
        y = ySet.T - (rotMat.dot(xSet[i].T) + t)

        # Find the norm of the difference
        yj = np.linalg.norm(y, axis=0)
        index = np.argmin(yj)

        # Check if the norm is within the acceptable range
        # If yes add the corresponding x and y points to the correspondance list
        if yj[index]<dMax:
            C.append((i,index))
            
    return C

In [11]:
def computeOptimalRigidRegistratrion(xSet,ySet,C):
  '''
    A function to find the optimal rotation and translation matrix that match xSet to ySet.

        Parameters:
            xSet (np.matrix): Point cloud
            ySet(np.matrix): Another Point cloud
            C (list): Correspondance list
        Return:
            R (list): Estimated optimal rotation matrix
            t (list): Estimated optimal translation matrix
    '''
  W = np.matrix([[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]])
  K = len(C)

  x = [xSet[i] for i,j in C]
  y = [ySet[j] for i,j in C]

  # Find the centroid of the X and Y set
  xCentroid = np.mean(x,axis=0)
  yCentroid = np.mean(y,axis=0)

  # Find the deviation of each points from its centroid
  xDeviation = np.subtract(x,xCentroid)
  yDeviation = np.subtract(y,yCentroid)

  # Compute the cross corelation matrix
  for i in range(K):
    yi = np.matrix([[yDeviation[i][0][0]],[yDeviation[i][0][1]],[yDeviation[i][0][2]]])   # Y matrix
    xiTrans = np.matrix([xDeviation[i][0][0],xDeviation[i][0][1],xDeviation[i][0][2]])    # X transpose matrix
    W += yi.dot(xiTrans)

  W = W/K # Cross corelation matrix

  U,s,VT = np.linalg.svd(W) # Singular Value Decomposition

  R = np.dot(U,VT)                        # Optimal rotation matrix
  t = yCentroid.T-np.dot(R,xCentroid.T)   # Optimal translation matrix
  return R,t
    

In [12]:
def ICP(xSet, ySet, transInit, rotMatInit, dMax, numICPIters ):
    '''
    A function to implement Iterative Closest Point to find the optimal rotation matirx that transforma xSet to ySet.

        Parameters:
            xSet (np.matrix): Point cloud
            ySet(np.matrix): Another Point cloud
            transInit (np.matrix): Initial guess of translation matrix
            rotMatInit (np.matrix): Initial guess of rotation matrix
            dMax (float): Maximum admissiable distance between each point in xSet and ySet
            numICPIters (int): Number of iterations
        Return:
            rotMat (list): Estimated optimal rotation matrix
            t (list): Estimated optimal translation matrix
    '''
    
    t = transInit
    rotMat = rotMatInit   

    # Repeat unitl convergence 
    for i in range(numICPIters):
        # Compute correspondence
        c = estimateCorrespondence(xSet,ySet,t,rotMat,dMax)
        # Compute and update rotation and translation matrix
        rotMat,t = computeOptimalRigidRegistratrion(xSet,ySet,c)
        
    return rotMat,t

ICP implementation with the following parameters:
1. Set the initial estimate for the optimal registration to the identity: (t0,R0) = (0, I3) ∈
SE(3).
2. Set dmax = .25
3. Set the number of ICP iterations to 30: num ICP iters = 30.

In [14]:
xSet = np.loadtxt("pclX.txt",dtype=float)
ySet = np.loadtxt("pclY.txt",dtype=float)

xSet = np.matrix(xSet)
ySet = np.matrix(ySet)

xFixed1 = []
xFixed2 = []
xFixed3 = []
yFixed1 = []
yFixed2 = []
yFixed3 = []
for i in range(len(xSet)):
    xFixed1.append(xSet[i,0])
    xFixed2.append(xSet[i,1])
    xFixed3.append(xSet[i,2])
for i in range(len(ySet)):
    yFixed1.append(ySet[i,0])
    yFixed2.append(ySet[i,1])
    yFixed3.append(ySet[i,2])

fig = go.Figure(data=[go.Scatter3d(x=xFixed1, y=xFixed2, z=xFixed3, mode='markers', marker=dict(size=1, color='red', opacity=1))])
fig.add_trace(go.Scatter3d(x=yFixed1, y=yFixed2, z=yFixed3, mode='markers', marker=dict(size=1, color='blue', opacity=1)))
fig.show()

In [15]:
dMax = 0.25
tInit = np.matrix([[0],[0],[0]])
rotMatInit = np.matrix([[1,0,0],[0,1,0],[0,0,1]])
numICPIters = 30

rotMat,t = ICP(xSet, ySet, tInit, rotMatInit, dMax, 30 )
final = []
for x in xSet:
        x = x.reshape(3,1)
        final.append(np.dot(rotMat,x) + t)
final = np.array(final)

The parameters (ˆt, ˆR) ∈ SE(3) for the estimated rigid transformation,

In [16]:
print("Rotation Matrix: \n",rotMat,"\nTranslation Matrix: \n",t)

Rotation Matrix: 
 [[ 0.95126601 -0.15043058 -0.26919069]
 [ 0.22323628  0.9381636   0.26460276]
 [ 0.21274056 -0.31180074  0.92602471]] 
Translation Matrix: 
 [[ 0.49661487]
 [-0.29392971]
 [ 0.29645004]]


In [17]:
xConverged = []
yConverged = []
zConverged = []
for i in range(len(final)):
    xConverged.append(final[i][0][0])
    yConverged.append(final[i][1][0])
    zConverged.append(final[i][2][0])


Red - Point cloud 1 (X set);
Blue - Point cloud 2 (Y set);
Green - Transformed X set

In [18]:
fig = go.Figure(data=[go.Scatter3d(x=xFixed1, y=xFixed2, z=xFixed3, mode='markers', marker=dict(size=1, color='red', opacity=1))])
fig.add_trace(go.Scatter3d(x=yFixed1, y=yFixed2, z=yFixed3, mode='markers', marker=dict(size=1, color='blue', opacity=1)))
fig.add_trace(go.Scatter3d(x=xConverged, y=yConverged, z=zConverged, mode='markers', marker=dict(size=1, color='green', opacity=1)))
fig.show()

Root Mean Square Error

In [19]:
finalCorrespondance = estimateCorrespondence(xSet,ySet,t,rotMat,dMax)

x = [xSet[i].T for i,j in finalCorrespondance]
y = [ySet[j].T for i,j in finalCorrespondance]

RMSE = 0
for i in range(len(finalCorrespondance)):

    RMSE += np.linalg.norm(y[i] - (np.dot(rotMat,x[i]) + t))

RMSE = math.sqrt(RMSE/len(finalCorrespondance))
print("Root Mean Square Error: ",RMSE)

Root Mean Square Error:  0.08898488534607683
